In [153]:
#Input
word = "climb"
word1 = "climb.01"
word2 = "LOCOMOTE-UP"

In [154]:
import jsontrips
import xml.etree.ElementTree as ET
ont = jsontrips.ontology()
def trips_roles(onttype):
    roles = []
    for role_dic in ont[onttype]['arguments']:
        roles.append([role_dic['role'] , role_dic['optionality']])
    return roles
def parse(root , rolesetid):
    if root == None:
        return
    for predicate in root:
        for roleset in predicate:
            if roleset.attrib['id'] == rolesetid:
                roles = extract_roles(roleset)
                #vnroles = extract_vnroles(roleset)
                break
        if roleset.attrib['id'] == rolesetid:
            break
    return roles
def extract_roles(roleset):
    roles_list = []
    vnroles = []
    for things in roleset:
        if things.tag == "roles":
            for role_thing in things:
                if role_thing.tag == "role":
                    roles_list.append([role_thing.attrib['n'],role_thing.attrib['f'],role_thing.attrib['descr']])
                    vnr = []
                    for vn in role_thing:
                        if vn.tag == 'vnrole':
                            vnr.append(vn.attrib['vntheta'])
                    vnroles.append(vnr)
            break
    return roles_list , vnroles

def get_trips_roles(c):
    l = []
    for i in range(len(c)):
        l.append(c[i][0])
    return set(l)

def present(num , a):
    for i in range(len(a)):
        if a[i][0] == num:
            return (True , i)
    return (False , 0)

def remove(listi , word):
    for i in range(len(listi)):
        if listi[i] == word:
            del listi[i]
            break
            
def from_vn(num , role , vnrole):
    dic = {'agent':['AGENT','EXPERIENCER'],'causer':['AGENT'],'instrument':['AGENT'],'stimulus':['AGENT','NEUTRAL','FORMAL'],
           'theme':['AGENT','AFFECTED','NEUTRAL','FORMAL'],'patient':['AFFECTED'],'destination':['AFFECTED'],'source':['AFFECTED' , 'SOURCE'],
           'experiencer':['AFFECTED','NEUTRAL','EXPERIENCER'],'product':['AFFECTED'],'result':['AFFECTED' , 'RESULT' , 'FORMAL'],'pivot':['AFFECTED'],
           'location':['NEUTRAL'],'topic':['NEUTRAL'],'attribute':['FORMAL'],'predicate':['FORMAL'],'beneficiary':['BENEFICIARY'],'recepient':['BENEFICIARY'], 'instrument':['METHOD'] , 'co-agent':['AGENT1']
          }
    if num == 1:
        if role == 'EXPERIENCER':
            l = dic[vnrole]
            remove(l,'EXPERIENCER')
            remove(l,'AGENT')
            remove(l,'AFFECTED')
            return l
        elif role == 'AGENT':
            l = dic[vnrole]
            remove(l,'EXPERIENCER')
            remove(l,'AGENT')
            return l
        
def get_list(num):
    if num == 'COM':
        return ['MANNER']
    elif num == 'LOC':
        return ['LOCATION']
    elif num == 'DIR':
        return ['RESULT' , 'SOURCE' , 'TRANSIENT-RESULT']
    elif num == 'GOL':
        return ['BENEFICIARY' , 'RESULT']
    elif num == 'MNR':
        return ['MANNER']
    elif num == 'TMP':
        return ['TIME']
    elif num == 'EXT':
        return ['EXTENT' , 'DEGREE']
    elif num == 'REC':
        return ['AFFECTED']
    elif num == 'PRD':
        return ['RESULT'] ## , 'FORMAL' , 'NEUTRAL']
    elif num == 'PRP':
        return ['REASON']
    elif num == 'CAU':
        return ['REASON']
    elif num == 'CXN':
        return ['COMPARE']
    else:
        return []
    
def role_matching(a,b,c):
    # Match ARG0 if present
    if present('0',a)[0]:
        if present('EXPERIENCER' , c)[0]:
            e = 'EXPERIENCER'
            print("ARG0 matched to EXPERIENCER")
        elif present('AGENT' , c)[0]:
            e = 'AGENT'
            print("ARG0 matched to AGENT")
        else:
            print("ARG0 is unmatched")
    if present('1',a)[0]:
        if 'thing' in a[present('1',a)[1]][2]:
            print("ARG1 is matched to AFFECTED")
        elif b[present('1',a)[1]]:
            p = get_trips_roles(c)
            l = set(from_vn(1 , e , b[present('1',a)[1]][0].lower()))
            p.intersection_update(l)
            print("ARG1 is matched to {}".format(p))   
        else:
            print("ARG1 is matched to AFFECTED")     ##PROVIDED THING + ED Ending Which have to code in 
    if present('2',a)[0]:
        if b[present('2',a)[1]]:
            l = get_list(a[present('2',a)[1]][1].upper())
            print("ARG2 is matched to intersection of {} and {}".format(from_vn(1 , 'AGENT',b[present('2',a)[1]][0].lower()),l))
        else:
            l = get_list(a[present('2',a)[1]][1].upper())
            print("ARG2 is matched to {}".format(l))

    if present('3',a)[0]:
        if b[present('3',a)[1]]:
            l = get_list(a[present('3',a)[1]][1].upper())
            print("ARG3 is matched to intersection of {} and {}".format(from_vn(1 , 'AGENT',b[present('3',a)[1]][0].lower()),l))
        else:
            l = get_list(a[present('3',a)[1]][1].upper())
            print("ARG3 is matched to {}".format(l))

    if present('4',a)[0]:
        if b[present('4',a)[1]]:
            l = get_list(a[present('4',a)[1]][1].upper())
            print("ARG4 is matched to intersection of {} and {}".format(from_vn(1 , 'AGENT',b[present('4',a)[1]][0].lower()),l))
        else:
            l = get_list(a[present('4',a)[1]][1].upper())
            print("ARG4 is matched to {}".format(l))

In [155]:
tree = ET.parse( 'frames/' + word + ".xml")
root = tree.getroot()
a, b = parse(root , word1)
c = trips_roles(word2)

In [156]:
role_matching(a,b,c)

ARG0 matched to AGENT
ARG1 is matched to AFFECTED
